### k_fold.ipynb
- Alessandro Trincone
- Mario Gabriele Carofano

> ...

In [ ]:
import auxfunc
import constants

import numpy as np
from artificial_neural_network import NeuralNetwork
from training_report import TrainingReport
import dataset_functions as df

import pprint
import time
from datetime import datetime
import copy

...

In [ ]:
def k_fold_cross_validation(
        Xtrain : list[np.ndarray],
        Ytrain : list[np.ndarray],
        k : int = constants.DEFAULT_K_FOLD_VALUE
) -> NeuralNetwork:
    
    """
        E' una tecnica di validazione che utilizza una parte indipendente del training set per la fase di validazione. Si utilizza per la selezione del modello che restituisce il minor errore di validazione.

        Parameters:
        -   Xtrain : la matrice di esempi da classificare.
        -   Ytrain : la matrice di etichette corrispondenti agli esempi (ground truth).
        -   k : e' un numero intero che indica in quante fold dividere il training set.

        Returns:
        -   net : la rete neurale che mostra la miglior configurazione di pesi e bias dopo le fasi di addestramento e validazione su tutte le fold del dataset.
    """

    history_report : list[TrainingReport] = []

    Xfolds, Yfolds = df.split_dataset(Xtrain, Ytrain, k)

    for i in range(k):

        print(f"\nFold {i+1} di {k}")

        net = NeuralNetwork(
            784, 64, 10,
            hidden_act_funs=auxfunc.leaky_relu,
            output_act_fun=auxfunc.identity,
            e_fun=auxfunc.cross_entropy_softmax,
            random_init=False
        )

        training_fold = np.concatenate([fold for j, fold in enumerate(Xfolds) if j != i])
        training_labels = np.concatenate([fold for j, fold in enumerate(Yfolds) if j != i])
        validation_fold = Xfolds[i]
        validation_labels = Yfolds[i]

        if i == 0:
            best_fold_params = {
                "Fold"      : i+1,
                "Weights"   : copy.deepcopy(net.weights),
                "Biases"    : copy.deepcopy(net.biases),
                "Report"    : copy.deepcopy(net.training_report),
                "History"   : copy.deepcopy(history_report)
            }

        history_report = net.train(training_fold, training_labels, validation_fold, validation_labels)

        # scelta dei parametri per la rete che mostrano il minor errore di validazione
        if i == 0 or net.training_report.validation_error < best_fold_params["Report"].validation_error:
            best_fold_params = {
                "Fold"      : i+1,
                "Weights"   : copy.deepcopy(net.weights),
                "Biases"    : copy.deepcopy(net.biases),
                "Report"    : copy.deepcopy(net.training_report),
                "History"   : copy.deepcopy(history_report)
            }

        if constants.DEBUG_MODE:
            break

    # end for i

    print(f"\nMiglior rete (fold): {best_fold_params['Fold']}")
    print(repr(best_fold_params["Report"]))

    # Disegno dei grafici delle curve di errore
    auxfunc.plot_error(
        [r.training_error for r in history_report],
        [r.validation_error for r in history_report]
    )

    # Disegno dei grafici delle curve di accuracy
    auxfunc.plot_accuracy(
        [r.training_accuracy for r in history_report],
        [r.validation_accuracy for r in history_report],
    )

    """
        Dopo aver individuato la configurazione di parametri (weights, biases) per cui il modello ottiene il minor errore di validazione, si riaddestra il modello sul training set completo.
    """
    net.weights = copy.deepcopy(best_fold_params["Weights"])
    net.biases = copy.deepcopy(best_fold_params["Biases"])
    history_report = net.train(Xtrain, Ytrain)

    # Disegno dei grafici delle curve di errore / accuracy
    auxfunc.plot_error([r.training_error for r in history_report])
    auxfunc.plot_accuracy([r.training_accuracy for r in history_report])

    net.save_network_to_file()

    return net

# end

...

In [ ]:
Xtrain, Ytrain, Xtest, Ytest = df.loadDataset(constants.COPPIE_TRAINING, constants.COPPIE_TEST)

...

In [ ]:
print(f"\nK-fold cross-validation iniziato: {datetime.now().strftime(constants.DATE_TIME_FORMAT)}")
start_time = time.time()

net = k_fold_cross_validation(Xtrain, Ytrain)

end_time = time.time()
tot_time = end_time - start_time

print(f"\nK-fold cross-validation completato: {datetime.now().strftime(constants.DATE_TIME_FORMAT)}")
print(f"Tempo trascorso: {tot_time:.3f} secondi")

Se si vuole utilizzare una configurazione di parametri di una rete già addestrata con accuracy del circa 80%.

In [ ]:
# net = NeuralNetwork.load_network_from_file(constants.OUTPUT_DIRECTORY+'nn.pkl')
# print(repr(net))

...

In [ ]:
# net.test(Xtest, Ytest, plot_mode=constants.PlotTestingMode.ALL)